In [1]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd 
import numpy as np
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1bukAvea564vEBxLeoSyQ4SIObU6RyVTx_SsQ-RF08Wc"
SAMPLE_RANGE_NAME = "A:N"
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])


In [2]:
df = pd.DataFrame(values[1:], columns=values[0])
df.head()

,no,place_id,location_lat,location_lng,name,types,street,ward,district,city,address,phone,open_hours,link
0,0,ChIJyxH9AwAtCzER3hy3L5vJUeY,11.0115453,106.4879277,kho phúc thịnh,parking,2F6Q+J57,Ấp Bàu Chứa,Củ Chi,Ho Chi Minh City,"2F6Q+J57, Ấp Bàu Chứa, Củ Chi, Thành phố Hồ Ch...",nan,nan,https://www.google.com/maps/search/?api=1&quer...
1,1008,ChIJn0prFa5pdTERVdPiwn-iyco,10.4039901,106.9222048,Bến Đò Hành Khách,parking,150 Hòa Hiệp,Long Hoà,Cần Giờ,Ho Chi Minh City,"150 Hòa Hiệp, Long Hoà, Cần Giờ, Thành phố Hồ ...",nan,"['Monday: Open 24 hours', 'Tuesday: Open 24 ho...",https://www.google.com/maps/search/?api=1&quer...
2,1007,ChIJ2d2Ozc85dTERz_ntQC8xi2s,10.596921,106.745636,Nhà Doanh,parking,192/9 tổ 9 ấp 3,Hiệp Phước,Nhà Bè,Ho Chi Minh City,"192/9 tổ 9 ấp 3, Hiệp Phước, Nhà Bè, Thành phố...",nan,nan,https://www.google.com/maps/search/?api=1&quer...
3,1006,ChIJCYa0Otg7dTERpe33_iYQx_o,10.625744,106.7502773,LÊ THÀNH GIÀU,parking,"Đường 19A, Lô EB9, Đường 19A, KCN",KCN,Nhà Bè,Ho Chi Minh City,"Lô EB9, Đường 19A, KCN, Nhà Bè, Thành phố Hồ C...",nan,nan,https://www.google.com/maps/search/?api=1&quer...
4,1005,ChIJy0-RV4Q7dTERIeLqabOwG4s,10.6337557,106.7428492,Sky35 vận tải,parking,35 Phan Văn Bảy,Hiệp Phước,Nhà Bè,Ho Chi Minh City,"35 Phan Văn Bảy, Hiệp Phước, Nhà Bè, Thành phố...",0918 133 739,"['Monday: Open 24 hours', 'Tuesday: Open 24 ho...",https://www.google.com/maps/search/?api=1&quer...


# DATA CONVERSION

In [3]:

def cast_dataframe_types(df):

    # df['no'] = df['no'].astype('String')
    # df['place_id'] = df['place_id'].astype('object')
    df['location_lat'] = pd.to_numeric(df['location_lat'], errors='coerce')
    df['location_lng'] = pd.to_numeric(df['location_lng'], errors='coerce')
    # df['name'] = df['name'].astype('String')
    df['types'] = df['types'].astype('category')
    # df['street'] = df['street'].astype('String')
    df['ward'] = df['ward'].astype('category')
    df['district'] = df['district'].astype('category')
    df['city'] = df['city'].astype('category')
    # df['address'] = df['address'].astype('String')
    # df['phone'] = df['phone'].astype('String')
    # df['open_hours'] = df['open_hours'].astype('String')
    # df['link'] = df['link'].astype('String')

    return df

In [4]:
def convert_string_nan_to_real_nan(df):
    return df.map(lambda x: np.nan if x == "nan" else x)

# OVERVIEW

In [5]:
df = convert_string_nan_to_real_nan(df)
df = cast_dataframe_types(df)
print("number of variables:" + str(len(df.columns)))
print("number of observation:" + str(df.shape[0]))
print("duplicate row: " +str(df.duplicated().sum()))
print("Total unique per col:")
print(df.nunique())
memory_usage = df.memory_usage(deep=True).sum()
print(f"Total memory usage of DataFrame: {memory_usage} bytes")


number of variables:14
number of observation:1010
duplicate row: 5
Total unique per col:
no              1005
place_id        1005
location_lat    1003
location_lng    1004
name             980
types              1
street           974
ward             206
district          24
city               1
address          988
phone            537
open_hours        91
link            1005
dtype: int64
Total memory usage of DataFrame: 1080080 bytes


In [6]:
print("Total missing cell:" + str(df.isna().sum().sum()))
print("Missing cell per col")
print(df.isna().sum())

Total missing cell:1015
Missing cell per col
no                0
place_id          0
location_lat      0
location_lng      0
name              0
types             0
street            1
ward              8
district          0
city              0
address           0
phone           454
open_hours      552
link              0
dtype: int64


In [7]:
print("Datatypes: ")
print(df.dtypes)

Datatypes: 
no                object
place_id          object
location_lat     float64
location_lng     float64
name              object
types           category
street            object
ward            category
district        category
city            category
address           object
phone             object
open_hours        object
link              object
dtype: object


# METADATA FILLING 

In [8]:
print(f"Number of Missing Ward Cell: {df['ward'].isna().sum()}")
print(f"Number of Missing District Cell: {df['district'].isna().sum()}")

Number of Missing Ward Cell: 8
Number of Missing District Cell: 0


In [9]:
import json
import pandas as pd
with open('metadata.json', 'r') as f:
    metadata = json.load(f)


In [10]:
ward_to_district = {}
district_to_city = {}
all_wards = set()
all_districts = set()
all_cities = set()

for province in metadata.values():
    city_name = province['FullName']
    city_name_en = province['FullNameEn']
    all_cities.add(city_name)
    all_cities.add(city_name_en)
    
    for district in province['District']:
        district_name = district['FullName']
        district_name_en = district['FullNameEn']
        district_to_city[district_name] = city_name
        district_to_city[district_name_en] = city_name_en
        all_districts.add(district_name)
        all_districts.add(district_name_en)
        
        for ward in district['Ward']:
            ward_name = ward['FullName']
            ward_name_en = ward['FullNameEn']
            ward_to_district[ward_name] = district_name
            ward_to_district[ward_name_en] = district_name_en
            all_wards.add(ward_name)
            all_wards.add(ward_name_en)

In [11]:
from langdetect import detect

def normalize_name_vn(name):
    name = name.lower()
    prefixes = ['xã ', 'phường ', 'quận ', 'huyện ', 'thành phố ']
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):]
            break
    return name.strip()
def normalize_name_en(name):
    name = name.lower()
    suffixes = ['district', 'city', 'county', 'town', 'village']
    for suffix in suffixes:
        if name.endswith(suffix):
            name = name[:-len(suffix)].strip()
            break
    return name.strip()

def detect_language(name):
    try:
        return detect(name)
    except:
        return 'unknown'

In [13]:
def fill_missing_data(row,district_to_city):
    lang = detect_language(row['name'])
    
    if lang == 'vi':
        if pd.isna(row['ward']) and pd.notna(row['district']):
            district_name = normalize_name_vn(row['district'])
            row['ward'] = next((ward for ward, district in ward_to_district.items()
                                if normalize_name_vn(district) == district_name or district.startswith(district_name)), None)
        
        if pd.isna(row['district']) and pd.notna(row['ward']):
            ward_name = normalize_name_vn(row['ward'])
            row['district'] = next((district for ward, district in ward_to_district.items()
                                    if normalize_name_vn(ward) == ward_name or ward.startswith(ward_name)), None)
        
        if pd.isna(row['city']) and pd.notna(row['district']):
            district_name = normalize_name_vn(row['district'])
            row['city'] = district_to_city.get(district_name, None)
    
    elif lang == 'en':
        if pd.isna(row['ward']) and pd.notna(row['district']):
            district_name = normalize_name_en(row['district'])
            row['ward'] = next((ward for ward, district in ward_to_district.items()
                                if normalize_name_vn((district) == district_name)), None)
        
        if pd.isna(row['district']) and pd.notna(row['ward']):
            ward_name = normalize_name_en(row['ward'])
            row['district'] = next((district for ward, district in ward_to_district.items()
                                    if ward == ward_name), None)
        
        if pd.isna(row['city']) and pd.notna(row['district']):
            district_name = normalize_name_en(row['district'])
            row['city'] = district_to_city.get(normalize_name_en(district_name), None)
    
    return row

In [16]:
df = df.apply(fill_missing_data, axis=1,args=(ward_to_district,district_to_city))


In [17]:
df.head()

,no,place_id,location_lat,location_lng,name,types,street,ward,district,city,address,phone,open_hours,link
0,0,ChIJyxH9AwAtCzER3hy3L5vJUeY,11.011545,106.487928,kho phúc thịnh,parking,2F6Q+J57,Ấp Bàu Chứa,Củ Chi,Ho Chi Minh City,"2F6Q+J57, Ấp Bàu Chứa, Củ Chi, Thành phố Hồ Ch...",NaN,NaN,https://www.google.com/maps/search/?api=1&quer...
1,1008,ChIJn0prFa5pdTERVdPiwn-iyco,10.403990,106.922205,Bến Đò Hành Khách,parking,150 Hòa Hiệp,Long Hoà,Cần Giờ,Ho Chi Minh City,"150 Hòa Hiệp, Long Hoà, Cần Giờ, Thành phố Hồ ...",NaN,"['Monday: Open 24 hours', 'Tuesday: Open 24 ho...",https://www.google.com/maps/search/?api=1&quer...
2,1007,ChIJ2d2Ozc85dTERz_ntQC8xi2s,10.596921,106.745636,Nhà Doanh,parking,192/9 tổ 9 ấp 3,Hiệp Phước,Nhà Bè,Ho Chi Minh City,"192/9 tổ 9 ấp 3, Hiệp Phước, Nhà Bè, Thành phố...",NaN,NaN,https://www.google.com/maps/search/?api=1&quer...
3,1006,ChIJCYa0Otg7dTERpe33_iYQx_o,10.625744,106.750277,LÊ THÀNH GIÀU,parking,"Đường 19A, Lô EB9, Đường 19A, KCN",KCN,Nhà Bè,Ho Chi Minh City,"Lô EB9, Đường 19A, KCN, Nhà Bè, Thành phố Hồ C...",NaN,NaN,https://www.google.com/maps/search/?api=1&quer...
4,1005,ChIJy0-RV4Q7dTERIeLqabOwG4s,10.633756,106.742849,Sky35 vận tải,parking,35 Phan Văn Bảy,Hiệp Phước,Nhà Bè,Ho Chi Minh City,"35 Phan Văn Bảy, Hiệp Phước, Nhà Bè, Thành phố...",0918 133 739,"['Monday: Open 24 hours', 'Tuesday: Open 24 ho...",https://www.google.com/maps/search/?api=1&quer...


In [18]:
print(f"Number of Missing Ward Cell: {df['ward'].isna().sum()}")
print(f"Number of Missing District Cell: {df['district'].isna().sum()}")

Number of Missing Ward Cell: 0
Number of Missing District Cell: 0


# RE-PROFILLING